# Mechanizmy uwagi i transformatory

Jednym z głównych ograniczeń sieci rekurencyjnych jest to, że wszystkie słowa w sekwencji mają taki sam wpływ na wynik. Powoduje to suboptymalną wydajność standardowych modeli LSTM typu encoder-decoder w zadaniach sekwencja na sekwencję, takich jak rozpoznawanie nazwanych jednostek czy tłumaczenie maszynowe. W rzeczywistości konkretne słowa w sekwencji wejściowej często mają większy wpływ na wyniki sekwencyjne niż inne.

Rozważmy model sekwencja na sekwencję, taki jak tłumaczenie maszynowe. Jest on realizowany za pomocą dwóch sieci rekurencyjnych, gdzie jedna sieć (**encoder**) kompresuje sekwencję wejściową do stanu ukrytego, a druga, **decoder**, rozwija ten stan ukryty w przetłumaczony wynik. Problem z tym podejściem polega na tym, że końcowy stan sieci ma trudności z zapamiętaniem początku zdania, co skutkuje niską jakością modelu w przypadku długich zdań.

**Mechanizmy uwagi** umożliwiają ważenie kontekstowego wpływu każdego wektora wejściowego na każdą prognozę wyjściową RNN. Realizuje się to poprzez tworzenie skrótów między stanami pośrednimi RNN wejściowego a RNN wyjściowego. W ten sposób, generując symbol wyjściowy $y_t$, uwzględniamy wszystkie stany ukryte wejścia $h_i$, z różnymi współczynnikami wagowymi $\alpha_{t,i}$. 

![Obraz przedstawiający model encoder-decoder z warstwą uwagi addytywnej](../../../../../lessons/5-NLP/18-Transformers/images/encoder-decoder-attention.png)
*Model encoder-decoder z mechanizmem uwagi addytywnej w [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), cytowany z [tego wpisu na blogu](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Macierz uwagi $\{\alpha_{i,j}\}$ reprezentuje stopień, w jakim określone słowa wejściowe wpływają na generowanie danego słowa w sekwencji wyjściowej. Poniżej znajduje się przykład takiej macierzy:

![Obraz przedstawiający przykładowe wyrównanie znalezione przez RNNsearch-50, zaczerpnięty z Bahdanau - arviz.org](../../../../../lessons/5-NLP/18-Transformers/images/bahdanau-fig3.png)

*Rysunek zaczerpnięty z [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Rys.3)*

Mechanizmy uwagi są odpowiedzialne za dużą część obecnych lub bliskich obecnym osiągnięć w przetwarzaniu języka naturalnego. Dodanie uwagi znacznie zwiększa jednak liczbę parametrów modelu, co prowadzi do problemów ze skalowaniem w przypadku RNN. Kluczowym ograniczeniem skalowania RNN jest to, że rekurencyjny charakter modeli utrudnia grupowanie i równoległe trenowanie. W RNN każdy element sekwencji musi być przetwarzany w kolejności sekwencyjnej, co oznacza, że nie można go łatwo zrównoleglić.

Zastosowanie mechanizmów uwagi w połączeniu z tym ograniczeniem doprowadziło do powstania obecnych modeli transformatorowych, które znamy i używamy dzisiaj, takich jak BERT czy OpenGPT3.

## Modele transformatorowe

Zamiast przekazywania kontekstu każdej poprzedniej prognozy do kolejnego kroku ewaluacji, **modele transformatorowe** wykorzystują **kodowania pozycyjne** i **uwagę**, aby uchwycić kontekst danego wejścia w ramach dostarczonego okna tekstowego. Poniższy obraz pokazuje, jak kodowania pozycyjne z uwagą mogą uchwycić kontekst w danym oknie.

![Animowany GIF pokazujący, jak przeprowadzane są ewaluacje w modelach transformatorowych.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Ponieważ każda pozycja wejściowa jest mapowana niezależnie na każdą pozycję wyjściową, transformatory mogą być lepiej równoleglone niż RNN, co umożliwia tworzenie znacznie większych i bardziej ekspresyjnych modeli językowych. Każda głowica uwagi może być używana do nauki różnych relacji między słowami, co poprawia zadania przetwarzania języka naturalnego.

## Budowanie prostego modelu transformatorowego

Keras nie zawiera wbudowanej warstwy transformatora, ale możemy zbudować własną. Jak wcześniej, skupimy się na klasyfikacji tekstu z wykorzystaniem zbioru danych AG News, ale warto wspomnieć, że modele transformatorowe osiągają najlepsze wyniki w bardziej wymagających zadaniach NLP.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Nowe warstwy w Keras powinny dziedziczyć klasę `Layer` i implementować metodę `call`. Zacznijmy od warstwy **Positional Embedding**. Skorzystamy z [kodu z oficjalnej dokumentacji Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Założymy, że wszystkie wejściowe sekwencje są uzupełniane do długości `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Ta warstwa składa się z dwóch warstw `Embedding`: jednej do osadzania tokenów (w sposób, o którym wcześniej rozmawialiśmy) oraz drugiej do osadzania pozycji tokenów. Pozycje tokenów są tworzone jako sekwencja liczb naturalnych od 0 do `maxlen` za pomocą `tf.range`, a następnie przekazywane przez warstwę osadzania. Dwa wynikowe wektory osadzeń są następnie dodawane, tworząc pozycyjnie osadzone reprezentacje wejścia o kształcie `maxlen`$\times$`embed_dim`.

Teraz zaimplementujmy blok transformera. Będzie on przyjmował wyjście wcześniej zdefiniowanej warstwy osadzania:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Transformer stosuje `MultiHeadAttention` do wejścia zakodowanego pozycyjnie, aby wygenerować wektor uwagi o wymiarze `maxlen`$\times$`embed_dim`, który następnie jest mieszany z wejściem i normalizowany za pomocą `LayerNormalization`.

> **Uwaga**: `LayerNormalization` jest podobne do `BatchNormalization`, omawianego w części *Computer Vision* tej ścieżki nauki, ale normalizuje wyjścia poprzedniej warstwy dla każdej próbki treningowej niezależnie, aby sprowadzić je do zakresu [-1..1].

Wyjście tej warstwy jest następnie przekazywane przez sieć `Dense` (w naszym przypadku - dwuwarstwowy perceptron), a wynik jest dodawany do końcowego wyjścia (które ponownie przechodzi normalizację).

Teraz jesteśmy gotowi, aby zdefiniować kompletny model transformera:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## Modele Transformer BERT

**BERT** (Bidirectional Encoder Representations from Transformers) to bardzo duża, wielowarstwowa sieć transformatorowa z 12 warstwami dla *BERT-base* i 24 dla *BERT-large*. Model jest najpierw wstępnie trenowany na dużym korpusie danych tekstowych (Wikipedia + książki) przy użyciu uczenia nienadzorowanego (przewidywanie zamaskowanych słów w zdaniu). Podczas wstępnego treningu model przyswaja znaczący poziom zrozumienia języka, który można następnie wykorzystać z innymi zbiorami danych poprzez dostrajanie. Ten proces nazywa się **uczeniem transferowym**.

![obrazek z http://jalammar.github.io/illustrated-bert/](../../../../../lessons/5-NLP/18-Transformers/images/jalammarBERT-language-modeling-masked-lm.png)

Istnieje wiele wariantów architektur Transformer, takich jak BERT, DistilBERT, BigBird, OpenGPT3 i inne, które można dostrajać.

Zobaczmy, jak możemy użyć wstępnie wytrenowanego modelu BERT do rozwiązania naszego tradycyjnego problemu klasyfikacji sekwencji. Skorzystamy z pomysłu i części kodu z [oficjalnej dokumentacji](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Aby załadować wstępnie wytrenowane modele, użyjemy **Tensorflow hub**. Najpierw załadujmy wektoryzator specyficzny dla BERT:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Ważne jest, aby użyć tego samego wektoryzatora, który został użyty podczas trenowania oryginalnej sieci. Wektoryzator BERT zwraca trzy komponenty:
* `input_word_ids`, czyli sekwencję numerów tokenów dla zdania wejściowego
* `input_mask`, wskazującą, która część sekwencji zawiera rzeczywiste dane wejściowe, a która jest wypełnieniem. Jest to podobne do maski generowanej przez warstwę `Masking`
* `input_type_ids` jest używane w zadaniach modelowania języka i pozwala określić dwa zdania wejściowe w jednej sekwencji.

Następnie możemy zainicjować ekstraktor cech BERT:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Warstwa BERT zwraca kilka przydatnych wyników:
* `pooled_output` to wynik uśrednienia wszystkich tokenów w sekwencji. Można to traktować jako inteligentne semantyczne osadzenie całej sieci. Jest to równoważne z wynikiem warstwy `GlobalAveragePooling1D` w naszym poprzednim modelu.
* `sequence_output` to wynik ostatniej warstwy transformera (odpowiada wynikowi `TransformerBlock` w naszym modelu powyżej).
* `encoder_outputs` to wyniki wszystkich warstw transformera. Ponieważ załadowaliśmy 4-warstwowy model BERT (co można łatwo zgadnąć z nazwy, która zawiera `4_H`), ma on 4 tensory. Ostatni z nich jest taki sam jak `sequence_output`.

Teraz zdefiniujemy model klasyfikacji end-to-end. Użyjemy *funkcjonalnej definicji modelu*, w której definiujemy wejście modelu, a następnie podajemy serię wyrażeń do obliczenia jego wyniku. Ustawimy również wagi modelu BERT jako nieuczestniczące w treningu i będziemy trenować jedynie końcowy klasyfikator:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Pomimo niewielkiej liczby trenowalnych parametrów, proces jest dość wolny, ponieważ ekstraktor cech BERT jest obciążeniem obliczeniowym. Wygląda na to, że nie udało nam się osiągnąć zadowalającej dokładności, albo z powodu braku treningu, albo z powodu ograniczonej liczby parametrów modelu.

Spróbujmy odblokować wagi BERT i również je wytrenować. Wymaga to bardzo małej szybkości uczenia oraz bardziej ostrożnej strategii treningowej z **warmup**, przy użyciu optymalizatora **AdamW**. Użyjemy pakietu `tf-models-official`, aby stworzyć optymalizator:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Jak widzisz, trening przebiega dość wolno - ale możesz spróbować poeksperymentować i przeprowadzić trening modelu przez kilka epok (5-10), aby sprawdzić, czy uda Ci się uzyskać lepszy wynik w porównaniu do podejść, które stosowaliśmy wcześniej.

## Biblioteka Huggingface Transformers

Innym bardzo popularnym (i nieco prostszym) sposobem korzystania z modeli Transformer jest [pakiet HuggingFace](https://github.com/huggingface/), który dostarcza proste elementy budulcowe do różnych zadań NLP. Jest on dostępny zarówno dla Tensorflow, jak i PyTorch, kolejnego bardzo popularnego frameworka sieci neuronowych.

> **Note**: Jeśli nie interesuje Cię, jak działa biblioteka Transformers - możesz przejść na koniec tego notatnika, ponieważ nie zobaczysz tu nic zasadniczo innego niż to, co zrobiliśmy wcześniej. Będziemy powtarzać te same kroki trenowania modelu BERT, używając innej biblioteki i znacznie większego modelu. Proces ten obejmuje dość długie treningi, więc możesz po prostu przejrzeć kod.

Zobaczmy, jak nasz problem można rozwiązać za pomocą [Huggingface Transformers](http://huggingface.co).


Pierwszą rzeczą, którą musimy zrobić, jest wybór modelu, którego będziemy używać. Oprócz kilku wbudowanych modeli, Huggingface zawiera [internetowe repozytorium modeli](https://huggingface.co/models), gdzie można znaleźć wiele więcej modeli wstępnie wytrenowanych przez społeczność. Wszystkie te modele można załadować i używać, podając jedynie nazwę modelu. Wszystkie wymagane pliki binarne dla modelu zostaną automatycznie pobrane.

Czasami może być konieczne załadowanie własnych modeli. W takim przypadku można określić katalog zawierający wszystkie odpowiednie pliki, w tym parametry dla tokenizer'a, plik `config.json` z parametrami modelu, wagi binarne itp.

Na podstawie nazwy modelu możemy zainicjalizować zarówno model, jak i tokenizer. Zacznijmy od tokenizer'a:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Obiekt `tokenizer` zawiera funkcję `encode`, która może być bezpośrednio użyta do kodowania tekstu:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Możemy również użyć tokenizera do zakodowania sekwencji w sposób odpowiedni do przekazania modelowi, tj. uwzględniając pola `token_ids`, `input_mask` itd. Możemy również określić, że chcemy tensory Tensorflow, podając argument `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

W naszym przypadku będziemy używać wstępnie wytrenowanego modelu BERT o nazwie `bert-base-uncased`. *Uncased* oznacza, że model jest niewrażliwy na wielkość liter.

Podczas trenowania modelu musimy dostarczyć tokenizowaną sekwencję jako wejście, dlatego zaprojektujemy proces przetwarzania danych. Ponieważ `tokenizer.encode` jest funkcją w Pythonie, zastosujemy tę samą metodę, co w poprzedniej jednostce, wywołując ją za pomocą `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Teraz możemy załadować rzeczywisty model za pomocą pakietu `BertForSequenceClassification`. Zapewnia to, że nasz model ma już wymaganą architekturę do klasyfikacji, w tym końcowy klasyfikator. Zobaczysz komunikat ostrzegawczy informujący, że wagi końcowego klasyfikatora nie są zainicjalizowane, a model wymaga wstępnego treningu - to jest całkowicie w porządku, ponieważ dokładnie to zamierzamy zrobić!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Jak widać z `summary()`, model zawiera prawie 110 milionów parametrów! Przypuszczalnie, jeśli chcemy wykonać proste zadanie klasyfikacji na stosunkowo małym zbiorze danych, nie chcemy trenować podstawowej warstwy BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Teraz możemy rozpocząć trening!

> **Uwaga**: Trenowanie pełnoskalowego modelu BERT może być bardzo czasochłonne! Dlatego przeszkolimy go tylko na pierwszych 32 partiach. To jedynie pokazuje, jak skonfigurować proces treningu modelu. Jeśli chcesz spróbować pełnoskalowego treningu - po prostu usuń parametry `steps_per_epoch` i `validation_steps`, i przygotuj się na długie oczekiwanie!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Jeśli zwiększysz liczbę iteracji, poczekasz wystarczająco długo i przeprowadzisz trening przez kilka epok, możesz oczekiwać, że klasyfikacja za pomocą BERT zapewni najlepszą dokładność! Dzieje się tak, ponieważ BERT już bardzo dobrze rozumie strukturę języka, a my musimy jedynie dostosować końcowy klasyfikator. Jednakże, ponieważ BERT jest dużym modelem, cały proces treningu zajmuje dużo czasu i wymaga znacznej mocy obliczeniowej! (GPU, a najlepiej więcej niż jednego).

> **Note:** W naszym przykładzie używamy jednego z najmniejszych wstępnie wytrenowanych modeli BERT. Istnieją większe modele, które prawdopodobnie dadzą lepsze rezultaty.


## Kluczowe informacje

W tej jednostce omówiliśmy najnowsze architektury modeli oparte na **transformerach**. Zastosowaliśmy je do naszego zadania klasyfikacji tekstu, ale podobnie modele BERT mogą być używane do ekstrakcji jednostek, odpowiadania na pytania i innych zadań związanych z NLP.

Modele transformerów reprezentują obecny stan wiedzy w dziedzinie NLP i w większości przypadków powinny być pierwszym rozwiązaniem, które warto wypróbować podczas wdrażania niestandardowych rozwiązań NLP. Jednak zrozumienie podstawowych zasad działania rekurencyjnych sieci neuronowych, omówionych w tym module, jest niezwykle ważne, jeśli chcesz budować zaawansowane modele neuronowe.



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczeniowej AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
